In [1]:
import json
import os
import random
import re
import time

In [2]:
import cloudscraper
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd
import h3

In [3]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [5]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Taco Bell Data

In [6]:
scraper = cloudscraper.create_scraper()

In [7]:
url_root = "https://locations.tacobell.com/
national_store_list = []

In [16]:
# Parse all locations in a state
for i in tqdm(range(len(states_df)), desc=f"Parsing States"):
    state_store_list = []
    state_code = states_df.iloc[i]["STUSPS"].lower()

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    state_url = f"{url_root}{state_code}.html"
    locations_r = scraper.get(state_url)

    if locations_r.status_code != 200:
        print(state_code)
        continue

    soup = BeautifulSoup(locations_r.text, "html.parser")

    city_as = soup.find_all("a", {"class": "Link DirLinks"})
    for city_a in tqdm(city_as, desc=f"Parsing Locations in {state_code}"):
        href = city_a.attrs["href"]

        city_url = f"{url_root}{href}.html"
        r_city = scraper.get(city_url)
        city_soup = BeautifulSoup(r_city.text, "html.parser")
        taco_bell_locations_as = city_soup.find_all(
            "a",
            {
                "class": "Link font-bold text-2xl text-purple underline uppercase hover:font-black"
            },
        )

        for taco_bell_locations_a in taco_bell_locations_as:
            taco_bell_url = taco_bell_locations_a.attrs["href"].replace("../", url_root)
            taco_bell_r = scraper.get(taco_bell_url)
            taco_bell_soup = BeautifulSoup(taco_bell_r.text, "html.parser")
            table = taco_bell_soup.find_all("table", {"class": "w-full"})
            if table:
                table = table[0]
                tds = table.find_all("td")
                times = list(set([td.find("span").text for td in tds[1::2]]))
                if len(times) == 1 and times[0] == "Closed":
                    print(taco_bell_r.url)
                    continue

            lat = float(
                taco_bell_soup.find("meta", {"itemprop": "latitude"}).attrs["content"]
            )
            lng = float(
                taco_bell_soup.find("meta", {"itemprop": "longitude"}).attrs["content"]
            )
            store_dict = {"STUSPS": state_code.upper(), "geometry": Point(lng, lat)}
            national_store_list.append(store_dict)
            state_store_list.append(store_dict)

        time.sleep(random.uniform(0.01, 0.5))

    if state_store_list:
        state_taco_bell = gpd.GeoDataFrame(state_store_list, crs=4326)
        state_taco_bell.to_file(f"data/states/{state_code}.gpkg")

Parsing States:   0%|          | 0/56 [00:00<?, ?it/s]

Parsing Locations in nc:   0%|          | 0/159 [00:00<?, ?it/s]

https://locations.tacobell.com/nc/clinton/1410-sunset-avenue.html
https://locations.tacobell.com/nc/williamston/1471-washington-street.html
https://locations.tacobell.com/nc/wilson/901-highway-301.html


Parsing Locations in ok:   0%|          | 0/76 [00:00<?, ?it/s]

https://locations.tacobell.com/ok/tinker-afb/3360-n-avenue.html


...

Parsing Locations in nd:   0%|          | 0/8 [00:00<?, ?it/s]

Parsing Locations in ar:   0%|          | 0/79 [00:00<?, ?it/s]

Parsing Locations in in:   0%|          | 0/137 [00:00<?, ?it/s]

Parsing Locations in mn:   0%|          | 0/72 [00:00<?, ?it/s]

Parsing Locations in ct:   0%|          | 0/43 [00:00<?, ?it/s]

In [18]:
taco_bells_state_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        state_taco_bell_gdf = gpd.read_file(constructed_path)
        taco_bells_state_gdfs.append(state_taco_bell_gdf)

In [20]:
taco_bell_gdf = gpd.GeoDataFrame(pd.concat(taco_bells_state_gdfs, ignore_index=True))

In [21]:
taco_bell_gdf = taco_bell_gdf.to_crs(9311)
taco_bell_gdf.to_file(f"data/stores.gpkg")

## Get Population Data

In [23]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [24]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [25]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

In [26]:
states_with_population_df = states_with_population_df.dropna()

## Merge Data

In [27]:
taco_bell_per_state_df = pd.Series(state_counts, name="stores")
taco_bell_per_state_df = taco_bell_per_state_df.reset_index()
taco_bell_per_state_df = taco_bell_per_state_df.rename(columns={"index": "STUSPS"})

In [29]:
taco_bell_per_state_df = states_with_population_df.merge(
    taco_bell_per_state_df, on="STUSPS", how="left"
)
taco_bell_per_state_df = taco_bell_per_state_df.fillna(0)

In [30]:
taco_bell_per_state_df["per_1000"] = taco_bell_per_state_df["stores"] / (
    taco_bell_per_state_df["POPULATION"] / 1000
)
taco_bell_per_state_df["per_10k"] = taco_bell_per_state_df["stores"] / (
    taco_bell_per_state_df["POPULATION"] / 10_000
)
taco_bell_per_state_df["per_100k"] = taco_bell_per_state_df["stores"] / (
    taco_bell_per_state_df["POPULATION"] / 100000
)
taco_bell_per_state_df["per_500k"] = taco_bell_per_state_df["stores"] / (
    taco_bell_per_state_df["POPULATION"] / 500_000
)
taco_bell_per_state_df["per_1m"] = taco_bell_per_state_df["stores"] / (
    taco_bell_per_state_df["POPULATION"] / 1_000_000
)
taco_bell_per_state_df["per_capita"] = (
    taco_bell_per_state_df["POPULATION"] / taco_bell_per_state_df["stores"]
)

In [31]:
taco_bell_per_state_df = taco_bell_per_state_df.to_crs(9311)

In [32]:
taco_bell_per_state_df.to_file("data/taco_bell_per_state.gpkg")